In [1]:
import os
import pandas as pd

In [2]:
dataset = 'FB15k-237'
models = ['SheafE_Multisection','SheafE_Diag','SheafE_Diag_Symmetric','TransE','HolE','StructuredEmbedding']
saveloc = '/home/gebhart/projects/sheaf_kg/data'
path = os.path.join(saveloc, dataset)

In [3]:
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and '.csv' in f]

In [4]:
files

['SheafE_Diag_Symmetric_64embdim_32sec_2norm_1000epochs_SoftplusLossloss_20210208-1313.csv',
 'SheafE_Multisection_64embdim_16esdim_1sec_2norm_1000epochs_SoftplusLossloss_20210208-1057.csv',
 'SheafE_Multisection_64embdim_64esdim_24sec_2norm_0.0001orthogonal_1000epochs_SoftplusLossloss_20210225-0903.csv',
 'TransE_1000epochs_128embdim_CrossEntropyLossloss_7seed_20210128-1228.csv',
 'SheafE_Multisection_64embdim_128esdim_1sec_2norm_1000epochs_SoftplusLossloss_20210208-1154.csv',
 'HolE_1000epochs_64embdim_CrossEntropyLossloss_42seed_20210128-1900.csv',
 'SheafE_Diag_64embdim_64sec_2norm_1000epochs_SoftplusLossloss_20210208-1209.csv',
 'SheafE_Multisection_64embdim_64esdim_24sec_2norm_1.0orthogonal_1000epochs_SoftplusLossloss_20210228-1539.csv',
 'SheafE_Multisection_64embdim_64esdim_24sec_2norm_0.3orthogonal_1000epochs_SoftplusLossloss_20210228-1538.csv',
 'HolE_1000epochs_64embdim_SoftplusLossloss_1234seed_20210126-1646.csv',
 'HolE_1000epochs_64embdim_CrossEntropyLossloss_1234seed_202

In [5]:
def_ret = {
          'model':None,
          'embdim':None,
          'esdim':None,
          'loss':None,
          'sec':None,
          'norm':None,
          'seed':None,
          'epochs':None,
          'orthogonal':None}
def parse_filename(f, def_ret=def_ret, models=models, path=path):
    ret = def_ret.copy()
    df = pd.read_csv(os.path.join(path,f))
    for model in models:
        if model in f:
            ret['model'] = model
            ret['filename'] = f
            # remove model name from string so later 
            # string matching doesn't get confused (<sec> <--> multi<sec>tion)
            f = f[f.find(model)+len(model):]
    if ret['model'] is None:
        return None
    for k in def_ret:
        if k in f:
            kix = f.find(k)
            usix = f.rfind('_', 0, kix)
#             print(k,f[usix+1:kix])
            ret[k] = f[usix+1:kix]
    
    dd = df[(df['Side'] == 'both') & (df['Type'] == 'avg')]
    ret['hits@10'] = dd[dd['Metric'] == 'hits_at_10'].Value.values[0]
    ret['hits@5'] = dd[dd['Metric'] == 'hits_at_5'].Value.values[0]
    ret['hits@3'] = dd[dd['Metric'] == 'hits_at_3'].Value.values[0]
    ret['hits@1'] = dd[dd['Metric'] == 'hits_at_1'].Value.values[0]
    ret['adjusted_mean_rank'] = dd[dd['Metric'] == 'adjusted_mean_rank'].Value.values[0]
    ret['mean_reciprocal_rank'] = dd[dd['Metric'] == 'mean_reciprocal_rank'].Value.values[0]
    return ret

In [6]:
df = []
for file in files:
    r = parse_filename(file)
    if r is not None:
        df.append(r)

In [9]:
df = pd.DataFrame.from_dict(df)
df.sort_values(by='mean_reciprocal_rank', ascending=False)

,model,embdim,esdim,loss,sec,norm,seed,epochs,orthogonal,filename,hits@10,hits@5,hits@3,hits@1,adjusted_mean_rank,mean_reciprocal_rank
3,TransE,128,None,CrossEntropyLoss,None,None,7,1000,None,TransE_1000epochs_128embdim_CrossEntropyLosslo...,0.325594,0.269009,0.228569,0.149158,0.197253,0.208623
38,TransE,128,None,CrossEntropyLoss,None,None,42,1000,None,TransE_1000epochs_128embdim_CrossEntropyLosslo...,0.325056,0.269522,0.228251,0.148229,0.200593,0.207846
32,TransE,64,None,CrossEntropyLoss,None,None,42,1000,None,TransE_1000epochs_64embdim_CrossEntropyLosslos...,0.290831,0.238795,0.202270,0.135752,0.214459,0.187305
48,TransE,64,None,CrossEntropyLoss,None,None,7,1000,None,TransE_1000epochs_64embdim_CrossEntropyLosslos...,0.290415,0.239236,0.202711,0.135091,0.225391,0.186937
28,TransE,64,None,CrossEntropyLoss,None,None,1234,1000,None,TransE_1000epochs_64embdim_CrossEntropyLosslos...,0.286256,0.236227,0.200533,0.134871,0.222968,0.185449
10,HolE,64,None,CrossEntropyLoss,None,None,1234,1000,None,HolE_1000epochs_64embdim_CrossEntropyLossloss_...,0.275908,0.223138,0.186099,0.119459,0.166666,0.172176
35,SheafE_Multisection,64,64,SoftplusLoss,64,2,None,1000,None,SheafE_Multisection_64embdim_64esdim_64sec_2no...,0.296971,0.228594,0.189500,0.106150,0.068807,0.171918
5,HolE,64,None,CrossEntropyLoss,None,None,42,1000,None,HolE_1000epochs_64embdim_CrossEntropyLossloss_...,0.269058,0.218906,0.184436,0.118358,0.173461,0.169098
45,HolE,64,None,CrossEntropyLoss,None,None,7,1000,None,HolE_1000epochs_64embdim_CrossEntropyLossloss_...,0.268935,0.218930,0.182087,0.115349,0.155805,0.168263
42,SheafE_Multisection,64,64,SoftplusLoss,128,2,None,1000,None,SheafE_Multisection_64embdim_64esdim_128sec_2n...,0.282586,0.215187,0.175555,0.110285,0.068762,0.167737
